
# Delta Sharing - consuming data using REST API

Let's deep dive on how Delta Sharing can be used to consume data using the native REST api.

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/delta-sharing/resources/images/delta-sharing-flow.png" width="900px"/>
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=governance&org_id=4214571749987147&notebook=%2F05-extra-delta-sharing-rest-api&demo_name=delta-sharing-airlines&event=VIEW&path=%2F_dbdemos%2Fgovernance%2Fdelta-sharing-airlines%2F05-extra-delta-sharing-rest-api&version=1">

## Exporing REST API Using Databricks OSS Delta Sharing Server

Databricks hosts a sharing server for test: https://sharing.delta.io/ 

*Note: it doesn't require authentification, real-world scenario require a Bearer token in your calls*

In [0]:
import requests
import json

data = requests.get('https://sharing.delta.io/delta-sharing/shares').json()
# Pretty-print the JSON data
print(json.dumps(data, indent=4))

In [0]:
data = requests.get('https://sharing.delta.io/delta-sharing/shares/delta_sharing/schemas').json()
# Pretty-print the JSON data
print(json.dumps(data, indent=4))

In [0]:
data = requests.get('https://sharing.delta.io/delta-sharing/shares/delta_sharing/schemas/default/tables').json()
# Pretty-print the JSON data
print(json.dumps(data, indent=4))

### Get metadata from our "boston-housing" table

In [0]:
data = requests.get('https://sharing.delta.io/delta-sharing/shares/delta_sharing/schemas/default/tables/boston-housing/metadata')
json_objects = data.text.strip().split('\n')
for obj in json_objects:
    print(json.dumps(json.loads(obj), indent=4))

### Getting the data
Delta Share works by creating temporary self-signed links to download the underlying files. It leverages Delta Lake statistics to pushdown the query and only retrive a subset of file. 

The REST API allow you to get those links and download the data:

In [0]:
import requests
import json

# Define the JSON payload
payload = {
    "predicateHints": [
        "date >= '2021-01-01'",
        "date <= '2021-01-31'"
    ],
    "limitHint": 1000
}

# Send the POST request
response = requests.post("https://sharing.delta.io/delta-sharing/shares/delta_sharing/schemas/default/tables/boston-housing/query", headers={"Content-Type": "application/json"}, json=payload)

json_objects = response.text.strip().split('\n')
for obj in json_objects:
    print(json.dumps(json.loads(obj), indent=4))